In [2]:



import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet


from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator



from sklearn.metrics import confusion_matrix, classification_report


# Задайте пути к папкам с данными
train_data_dir = 'f:/Python/Studies/Project/My/img/train_big'
validation_data_dir = 'f:/Python/Studies/Project/My/img/test_new_1'
test_data_dir = 'f:/Python/Studies/Project/My/img/test_new_1'




In [5]:

# Задайте параметры модели
input_shape = (224, 224, 3)
batch_size = 64
epochs = 40
num_classes = 3

# Задайте аугментацию для тренировочных изображений
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Загрузите тренировочные изображения и примените аугментацию
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Загрузите валидационные изображения и выполните нормализацию
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size= input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)





Found 12194 images belonging to 3 classes.
Found 1530 images belonging to 3 classes.
Found 1530 images belonging to 3 classes.


In [3]:

# Загрузите предобученную модель MobileNet без верхних слоев
conv_base = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)



# # Размораживаем
conv_base.trainable = True

    
  

# # Конструируем модель
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))


In [4]:
# Скомпилируем модель
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

es_callback = EarlyStopping(patience=10, verbose=1, restore_best_weights=True)


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[es_callback]
)

# Сохраните модель
model.save('MobileNet_f1_0.h5')

In [6]:
predict_pred_MN = model.predict(validation_generator)
simple_seq_preds = predict_pred_MN.argmax(axis=1)

print(classification_report(validation_generator.classes, simple_seq_preds))

24/24 [==============================] - 9s 253ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       477
           1       0.99      0.98      0.99       540
           2       0.98      0.99      0.99       513

    accuracy                           0.99      1530
   macro avg       0.99      0.99      0.99      1530
weighted avg       0.99      0.99      0.99      1530



In [7]:
cm = confusion_matrix (validation_generator.classes, simple_seq_preds)
cm

array([[473,   2,   2],
       [  3, 530,   7],
       [  3,   2, 508]], dtype=int64)

In [21]:
validation_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2])